In [160]:
import os, sys
import json
import random
import cv2

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import make_scorer

module_path = os.path.abspath(os.path.join("..", ".."))
sys.path.append(module_path)

from dataset_utils.Dataset import Dataset
from dataset_utils.config import DS_ROOT
from dataset_utils.utils import imshow_by_path, imshow_by_name, imshow

from dataset_utils.Dataset import Dataset
from dataset_utils.utils import extract_eyes_region_from_aligned_face, gray, eqhist
from dataset_utils.utils import extract_eye_regions_from_aligned_face

import data_provider

## noise detectorの学習

In [140]:
ANT_PATH = "./flmk_annotation.json"
DATA_DIR = os.path.join(DS_ROOT, "aligned_with_68")
data = data_provider.get_data(DATA_DIR, ANT_PATH)

In [52]:
data =  random.sample(data, len(data))

train_d = [d for d in data if d.pid > 3]
test_d = [d for d in data if d.pid <= 3]
# train_d = data[test_num:]

x_train = [d.x for d in train_d]
x_test = [d.x for d in test_d]
y_train = [d.label for d in train_d]
y_test = [d.label for d in test_d]

In [53]:
print("dimension: %d" % x_train[0].shape[0])

print("noise num: %d" % len([d for d in data if d.label == 1]))
print("other num: %d" % len([d for d in data if d.label == 0]))

print("train noise num: %d" % len([y for y in y_train if y == 1]))
print("train other num: %d" % len([y for y in y_train if y == 0]))
print("test noise num: %d" % len([y for y in y_test if y == 1]))
print("test other num: %d" % len([y for y in y_test if y == 0]))

dimension: 408
noise num: 727
other num: 3100
train noise num: 609
train other num: 2703
test noise num: 118
test other num: 397


In [54]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=256)
# pca.fit(x_train)
# transformed = pca.fit_transform(x_train)
# print('各次元の寄与率: {0}'.format(pca.explained_variance_ratio_))
# print('累積寄与率: {0}'.format(sum(pca.explained_variance_ratio_)))

In [55]:
classifier = RandomForestClassifier(max_depth=15)
# classifier = RandomForestClassifier()
classifier.fit(x_train, y_train)
print("train score: %f" % classifier.score(x_train, y_train))
print("test score: %f" % classifier.score(x_test, y_test))
print("f1 score: %f" % f1_score(y_test, classifier.predict(x_test), pos_label=1))
print("recall: %f" % recall_score(y_test, classifier.predict(x_test), pos_label=1))
print("precision: %f" % precision_score(y_test, classifier.predict(x_test), pos_label=1))
print("confusion matrix: \n%s" % confusion_matrix(y_test, classifier.predict(x_test)))

train score: 0.970713
test score: 0.867961
f1 score: 0.630435
recall: 0.491525
precision: 0.878788
confusion matrix: 
[[389   8]
 [ 60  58]]


## noise detectorを利用してデータセットのノイズを検出してjsonにファイル名を書き出す

In [42]:
data =  random.sample(data, len(data))
data = data_provider.get_data(DATA_DIR, ANT_PATH)
X = [d.x for d in data]
Y = [d.label for d in data]
classifier = RandomForestClassifier(max_depth=15)
classifier.fit(X, Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [43]:
noise_dict = {}
dataset = Dataset(DATA_DIR)
data = dataset.data
for i, d in enumerate(data):
    if d.glasses:
        continue
    if d.id % 5 != 0:
        continue
    img = cv2.imread(d.path)
    img = eqhist(gray(extract_eyes_region_from_aligned_face(img)))
    x = img.flatten()
    y = classifier.predict([x])
    if y == 1:
#         imshow_by_path(d.path)
        noise_dict[d.name] = 1
print("complete.")

complete.


In [44]:
with open(os.path.join(DS_ROOT, "predicted_noise.json"), "w") as fw:
    json.dump(noise_dict, fw)

## 新たに作成した片目ずつのアノテーションファイルからデータセットを構築する
- 右目は反転してバリエーションを減らす

In [31]:
ANT_PATH = "./annotations_new.json"
with open(ANT_PATH, "r") as fr:
    ant = json.load(fr)


In [404]:
eye_dir = "aligned_with_68"
dataset = Dataset(os.path.join(DS_ROOT, "aligned_with_68"))

pos = []
neg = []

class Sample():
    def __init__(self, x, img, label, pid):
        self.x = x
        self.img = img
        self.label = label
        self.pid = pid
        
def each_eye_path(path):
    ext = path.split(".")[1]
    left =  path.split(".")[0] + "_left" + "." + ext
    right =  path.split(".")[0] + "_right" + "." + ext
    return left, right

def get_samples():
    s = []
    for d in dataset.data:
        if d.glasses:
            continue
        if d.id % 5 != 0 or d.id > 50:
            continue

        lname, rname = each_eye_path(d.name)
        img = cv2.imread(d.path)
        limg, rimg = extract_eye_regions_from_aligned_face(img, margin=6)
        limg, rimg = [eqhist(gray(limg)) for img in [limg, rimg]]
        rimg = cv2.flip(rimg, 1) # 右目を左右反転

        if lname not in ant:
            s.append(Sample(limg.flatten(), limg, 0, d.pid))
        elif ant[lname] == "ok":
            s.append(Sample(limg.flatten(), limg, 0, d.pid))
        elif ant[lname] in  {"big_gap"}:
            s.append(Sample(limg.flatten(), limg, 1, d.pid))

        if rname not in ant:
            s.append(Sample(rimg.flatten(), rimg, 0, d.pid))
        elif ant[rname] == "ok":
            s.append(Sample(rimg.flatten(), rimg, 0, d.pid))
        elif ant[rname] in  {"big_gap"}:
            s.append(Sample(rimg.flatten(), rimg, 1, d.pid))
    return s

In [405]:
samples = get_samples()
pos = [s for s in samples if s.label == 1]
neg = [s for s in samples if s.label == 0]
print("pos num: %d" % len(pos))
print("neg num: %d" % len(neg))

neg = random.sample(neg, len(pos))
samples = neg + pos
samples = random.sample(samples, len(samples))

pos num: 534
neg num: 6585


# validation

In [406]:
test = [s for s in samples if s.pid < 7]
train = [s for s in samples if s.pid >= 7]

x_train = [s.x for s in train]
y_train  = [s.label for s in train]
x_test = [s.x for s in test]
y_test  = [s.label for s in test]

classifier = RandomForestClassifier(max_depth=15)
# classifier = GradientBoostingClassifier()
# classidier = SVC()
classifier.fit(x_train, y_train)
print("train score: %f" % classifier.score(x_train, y_train))
print("confusion matrix: \n%s" % confusion_matrix(y_train, classifier.predict(x_train)))
print("test score: %f" % classifier.score(x_test, y_test))
print("f1 score: %f" % f1_score(y_test, classifier.predict(x_test), pos_label=1))
print("recall: %f" % recall_score(y_test, classifier.predict(x_test), pos_label=1))
print("precision: %f" % precision_score(y_test, classifier.predict(x_test), pos_label=1))
print("confusion matrix: \n%s" % confusion_matrix(y_test, classifier.predict(x_test)))

train score: 0.997344
confusion matrix: 
[[351   0]
 [  2 400]]
test score: 0.879365
f1 score: 0.861314
recall: 0.893939
precision: 0.830986
confusion matrix: 
[[159  24]
 [ 14 118]]


# train -> ノイズ検出

In [407]:
samples = random.sample(samples, len(samples))
X = [s.x for s in samples]
Y = [s.label for s in samples]
classifier = RandomForestClassifier(max_depth=15)
classifier.fit(x_train, y_train)
print("train score: %f" % classifier.score(x_train, y_train))

train score: 0.992032


In [413]:
noise_dict = {}
DATA_DIR = os.path.join(DS_ROOT, "aligned_with_68")
dataset = Dataset(DATA_DIR)
data = dataset.data
for i, d in enumerate(data[:100]):
    if d.glasses:
        continue
    if d.id % 5 != 0:
        continue
    img = cv2.imread(d.path)
    limg, rimg = extract_eye_regions_from_aligned_face(img, margin=6)
    limg, rimg = [eqhist(gray(limg)) for img in [limg, rimg]]
    rimg = cv2.flip(rimg, 1) # 右目を左右反転
    left = limg.flatten()
    right = rimg.flatten()
    y_left, y_right = classifier.predict([left]), classifier.predict([right])
    if y_left == 1 or y_right == 1:
        noise_dict[d.name] = 1

print("complete.")

complete.


In [421]:
print(len(noise_dict))

3925


In [416]:
with open(os.path.join(DS_ROOT, "predicted_noise_each_eye.json"), "w") as fw:
    json.dump(noise_dict, fw)